## Generates statistics for FOF-CT datasets

### Generates excel file with columns: 
#### year, project, no_papers, no_datasets, no_files, cumulative_papers, cumulative_datasets, cumulative_files

## Instructions:
1. Log in to https://data.4dnucleome.org
2. Go to link: https://data.4dnucleome.org/browse/?experiments_in_set.experiment_type.display_title=DNA+FISH&experiments_in_set.experiment_type.display_title=multiplexed+FISH&experiments_in_set.experiment_type.display_title=RNA+FISH&experimentset_type=replicate&type=ExperimentSetReplicate
3. Select status = 'released'
4. Press "Select All"
5. Under "all file types" select all FOF-CT* file types
6. Download the metadata file copy the "Experiment Accession" column and unique the accessions and use it as a sets_list

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import get_workflow_details, delete_wfrs
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
my_auth = get_key('', keyfile='')
graph_stats_outfile = "" #Add name of output file

In [ ]:
sets_list = []

if sets_list:
    esets = [ff_utils.get_metadata(i, my_auth) for i in sets_list]
    
    print("No. of objects {}".format(len(esets)))

In [ ]:
#getting expset metadata : no. of processed files, associated publication ref
files_per_expset = []
pubs_id = []
pubs_ref = []
expset_acc = []
na = 1


for eset in esets:
    eset_acc = eset.get('accession')
    expset_acc.append(eset_acc)
    total_files = []
    file_types = []
    if eset.get('processed_files'):
        proc_files = eset.get('processed_files')
        for file in proc_files:
            file_type = file.get('file_type')
            file_acc = file.get('accession')
            total_files.append(file_acc)
            file_types.append(file_type)       
    else:        
        exps = eset.get('experiments_in_set')
        for exp in exps:
            if exp.get('processed_files'):
                proc_files_inexp = exp.get('processed_files')
                for file in proc_files_inexp:
                    file_type = file.get('file_type')
                    file_acc = file.get('accession')
                    total_files.append(file_acc)
                    file_types.append(file_type)
            else:
                print(eset_acc, 'does not have any procfiles')
    files_per_expset.append(len(total_files))
    for ftype in file_types:
        if 'FOF' not in ftype:
            print(eset_acc, 'this expset has proc files other than fof-ct')
    if eset.get('produced_in_pub'):     
        pub = eset.get('produced_in_pub')
        pubs_id.append(pub.get('@id'))
        pubs_ref.append(pub.get('uuid'))
    else:
        eset_public_release = eset.get('public_release')
        eset_award = eset.get('award')
        eset_project = eset_award.get('project')
        pub_id = 'N/A' + ' released on ' + eset_public_release + ' ' + eset_project
        pubs_id.append(pub_id)
        pubs_ref.append(pub_id)
                

In [ ]:
# adding lists to dictionary and then to datafram
expset_dic = {}
expset_dic['expset_acc']=expset_acc
expset_dic['publication_ref']=pubs_ref
expset_dic['total_files']=files_per_expset


stats_df = pd.DataFrame.from_dict(expset_dic)

#Aggregating the table based on publication id by counting the number of expsets and summing the total no. of files for each expset
new = stats_df.groupby('publication_ref').agg({'expset_acc': 'count', 'total_files': 'sum'})

#Get a list of publication references
pub_refs = new.index.tolist()

In [ ]:
#getting publication metadata

refs = []
ids=[]
released_date =[]
published_date=[]
project=[]
data_portal_ref = []

for ref in pub_refs:
    if ref not in refs:
        if 'N/A' not in ref:
            refs.append(ref)
            pub_metadata = ff_utils.get_metadata(ref, my_auth)
            ids.append(pub_metadata.get('ID'))
            data_portal_id = pub_metadata.get('@id').split('/')[2]
            data_portal_ref.append(data_portal_id)
            released_date.append(pub_metadata.get('public_release'))
            published_date.append(pub_metadata.get('date_published'))
            award = pub_metadata.get('award')
            project.append(award.get('project'))
        else:
            ids.append('N/A')
            released_date.append(ref.split(' ')[3])
            published_date.append('N/A')
            project.append(ref.split(' ')[4])
            data_portal_ref.append(ref)

In [ ]:
# adding lists to dictionary and then to dataframe, merging two dataframes and calculating cumulative values
paper_dic = {}
paper_dic['ids']=ids
paper_dic['released_date']=released_date
paper_dic['published_date']=published_date
paper_dic['project']=project
paper_dic['publication_ref']=data_portal_ref

paper_df = pd.DataFrame.from_dict(paper_dic)

# merging two dataframes together based on pub_id

merged = pd.merge(paper_df, new, on="publication_ref", how="outer" )
merged['year'] = merged['released_date'].str.split('-', expand=True)[0]
merged.to_excel('/Users/rahinavelkar/Desktop/fof-ct-stats/updated-new2.xlsx', index=False)

final = merged.groupby(['year','project']).agg({'publication_ref':'count','expset_acc': 'sum', 'total_files': 'sum'}).reset_index()
final.to_excel('/Users/rahinavelkar/Desktop/fof-ct-stats/updated_final.xlsx')

final.loc[9] = ['2023', 'External', 0,0,0]
final_sorted = final.sort_values(by='year', ignore_index=True).reset_index()
final_sorted

final_sorted.to_excel('/Users/rahinavelkar/Desktop/fof-ct-stats/updated_final.xlsx', )

df = pd.read_excel('/Users/rahinavelkar/Desktop/fof-ct-stats/updated_final.xlsx')
fourdn = df[df["project"] == "4DN"].copy()

# Calculate the cumulative sum for the filtered DataFrame
fourdn['cumulative_papers'] = fourdn["publication_ref"].cumsum()
fourdn['cumulative_datasets'] = fourdn["expset_acc"].cumsum()
fourdn['cumulative_files'] = fourdn["total_files"].cumsum()

# Update original DataFrame with cumulative sum values, where condition is met
df.loc[df["project"] == "4DN", 'cumulative_papers'] = fourdn['cumulative_papers'].values
df.loc[df["project"] == "4DN", 'cumulative_datasets'] = fourdn['cumulative_datasets'].values
df.loc[df["project"] == "4DN", 'cumulative_files'] = fourdn['cumulative_files'].values

external = df[df["project"] == "External"].copy()

# Calculate the cumulative sum for the filtered DataFrame
external['cumulative_papers'] = external["publication_ref"].cumsum()
external['cumulative_datasets'] = external["expset_acc"].cumsum()
external['cumulative_files'] = external["total_files"].cumsum()

# Update original DataFrame with cumulative sum values, where condition is met
df.loc[df["project"] == "External", 'cumulative_papers'] = external['cumulative_papers'].values
df.loc[df["project"] == "External", 'cumulative_datasets'] = external['cumulative_datasets'].values
df.loc[df["project"] == "External", 'cumulative_files'] = external['cumulative_files'].values

# Update original DataFrame with cumulative sum values, where condition is met
#df.loc[df["project"] == "External", 'Cumulative Sum'] = external['Cumulative Sum'].values

df["cumulative_papers"] = df["cumulative_papers"].astype(int)
df["cumulative_datasets"] = df["cumulative_datasets"].astype(int)
df["cumulative_files"] = df["cumulative_files"].astype(int)

df.to_excel(graph_stats_outfile)

### Optional: To generate plot year vs papers by overlaying datasets and files information

In [ ]:
# Pivoting data for stacked bar chart
df_pivot = df.pivot(index="year", columns="project", values="Cumulative Sum").fillna(0)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))

# Stacked bar chart
df_pivot.plot(kind="bar", stacked=True, ax=ax, colormap="Set2")  #change color here, color map options in the last block


#create separate dataframes for plot lines

df_intfiles = df[df['project']=='4DN']
plot_intfiles = df_intfiles[['year','datasets','files']].reset_index()

df_extfiles = df[df['project']=='External']
plot_extfiles = df_extfiles[['year','datasets','files']].reset_index()


#creating cumsum for files and datasets
plot4dn = generate_cumsum_datasets(plot_intfiles)
plotext = generate_cumsum_datasets(plot_extfiles)

# Overlay line plots for 'datasets' and 'files'
ax2 = ax.twinx()
ax2.plot(plot4dn['cumsum_datasets'], color="red", marker="o", linestyle="-", label="4DN datasets")  #change color here
ax2.plot(plot4dn['cumsum_files'], color="maroon", marker="o", linestyle="-", label="4DN files") #change color here
ax2.plot(plotext['cumsum_datasets'], color="purple", marker="s", linestyle="--", label="External datasets") #change color here
ax2.plot(plotext['cumsum_files'], color="blue", marker="s", linestyle="--", label="External files") #change color here

# Labels and title
ax.set_xlabel("Year")
ax.set_ylabel("Number of Papers (cumulative)")
ax2.set_ylabel("Number of Datasets & Files (cumulative)")
ax.set_title("FOF-CT adopted papers deposited in 4DN")

# Legends
ax.legend(title="Project", loc="upper left")
ax2.legend(title="No. of files and datasets (cumulative)", bbox_to_anchor=(1.55, 1), fancybox=True)
plt.figure(figsize=(10, 6))

plt.savefig('/Users/rahinavelkar/Desktop/fof-stats.png', dpi=199)

# Display plot
plt.show()